In [ ]:
import pandas as pd
import numpy as np

In [ ]:
BASIC_PURCHASE_INFO = {
    'property_value':  650000,
    'purchase_price':  650000
}

In [ ]:
MORTGAGE_INFO = {
    'interest_rate': 0.031, # %
    'down_payment': 0.25, # %
    'loan_term': 30
}

In [ ]:
# initialize variables

BASIC_PURCHASE_INFO['down_payment'] = BASIC_PURCHASE_INFO['purchase_price'] * MORTGAGE_INFO['down_payment']
BASIC_PURCHASE_INFO['equity_at_purchase'] = BASIC_PURCHASE_INFO['property_value'] - BASIC_PURCHASE_INFO['purchase_price'] + BASIC_PURCHASE_INFO['down_payment']

MORTGAGE_INFO['loan_amount'] = BASIC_PURCHASE_INFO['purchase_price'] * (1 - MORTGAGE_INFO['down_payment'])
MORTGAGE_INFO['monthly_payment'] = 2081.70 # hardcoded for now
MORTGAGE_INFO['annual_payment'] = MORTGAGE_INFO['monthly_payment'] *12

In [ ]:
CALC_ASSUM = {
    'rental_income': 2700,
    'vacancy': 0,
    'maintenance_reserve': 0,
    'management_fee': 0.15, # %
    'monthly_property_tax': 167.00,
    'monthly_insurance': 50.00,
    'utilities': {
        'monthly_electricity': 30,
        'monthly_gas': 30,
        'monthly_water': 30
    },
    'rental_income_increase': 2, #%
    'property_tax_increase': 2,  #%
    'closing_cost_buy': 0.02, #%
    'closing_cost_sell': 0
}

BASIC_PURCHASE_INFO['total_cash_required'] = BASIC_PURCHASE_INFO['down_payment'] + CALC_ASSUM['closing_cost_buy']*MORTGAGE_INFO['loan_amount']

In [ ]:
BASIC_PURCHASE_INFO['total_cash_required']

In [ ]:
def gross_scheduled_income(year):
    return round(12*CALC_ASSUM['rental_income'] * (1+CALC_ASSUM['rental_income_increase']/100)**((year * 12 - 12 ) / 12), 2)

In [ ]:
def property_tax(year):
    return round(12*CALC_ASSUM['monthly_property_tax']*(1+CALC_ASSUM['property_tax_increase']/100) ** ((year * 12 - 12 ) / 12), 2)

In [ ]:
def generate_years():
    return range(1,31)

In [ ]:
# calculating income

gross_income = np.array([gross_scheduled_income(i) for i in generate_years()])

total_operating_income = gross_income + 0

In [ ]:
# calculate operating expenses

property_taxes = np.array([property_tax(i) for i in generate_years()])
insurance = np.full(max(generate_years()), CALC_ASSUM['monthly_insurance']*12)

# calculate total utilities
total_util = 0
for util in CALC_ASSUM['utilities'].keys():
    total_util += CALC_ASSUM['utilities'][util]

utility = np.full(max(generate_years()), total_util*12)
property_management = gross_income * CALC_ASSUM['management_fee'] 

total_operating_expense = property_taxes + insurance + utility + property_management

In [ ]:
property_management

In [ ]:
total_operating_expense

In [ ]:
net_operating_income = total_operating_income - total_operating_expense

In [ ]:
annual_cash_flow = net_operating_income - MORTGAGE_INFO['annual_payment']
monthly_cash_flow = annual_cash_flow / 12

In [ ]:
# calculate financial performance

In [ ]:
cap_rate = net_operating_income / BASIC_PURCHASE_INFO['property_value']

In [ ]:
cash_on_cash = np.round((annual_cash_flow / BASIC_PURCHASE_INFO['total_cash_required'])*100, 2)

In [ ]:
cash_on_cash

In [ ]:
-np.pmt(MORTGAGE_INFO['interest_rate'] / 12, MORTGAGE_INFO['loan_term']*12, MORTGAGE_INFO['loan_amount'])

In [ ]:
def cumulative_principal(start, end, rate, nper, pv):

    cp = 0
    for i in range(start,end+1):
        cp = cp + (-np.ppmt(rate / 12, i, nper*12, pv))
    
    return cp

In [ ]:
principal_paydown = np.array([cumulative_principal(i*12-11, i*12, MORTGAGE_INFO['interest_rate'], MORTGAGE_INFO['loan_term'], MORTGAGE_INFO['loan_amount']) for i in generate_years()])

In [ ]:
mortgage_balance = np.array([round(np.fv(MORTGAGE_INFO['interest_rate'] / 12, i*12, 2081.7, -MORTGAGE_INFO['loan_amount']), 2) for i in generate_years()])

In [ ]:
mortgage_balance

In [ ]:
equity_wealth = BASIC_PURCHASE_INFO['property_value'] - mortgage_balance

In [ ]:
effective_net_cash_flow = annual_cash_flow + principal_paydown

In [ ]:
return_on_equity = effective_net_cash_flow / equity_wealth

In [ ]:
plus_sales_proceeds_if_final_year = annual_cash_flow + equity_wealth - BASIC_PURCHASE_INFO['property_value']*CALC_ASSUM['closing_cost_sell']

In [ ]:
accumulated_cash_flow = np.cumsum(annual_cash_flow)

In [ ]:
accumulated_cash_flow

In [ ]:
annual_cash_flow

In [ ]:
effective_future_value = plus_sales_proceeds_if_final_year + accumulated_cash_flow + annual_cash_flow

In [ ]:
effective_future_value

In [ ]:
annualized_return = effective_future_value / BASIC_PURCHASE_INFO['total_cash_required']

In [ ]:
annualized_return = effective_future_value / BASIC_PURCHASE_INFO['total_cash_required']

for i in range(0,len(annualized_return)):
    annualized_return[i] = annualized_return[i]**(1/(i+1))-1


In [ ]:
annualized_return

In [ ]:
return_on_investment = (plus_sales_proceeds_if_final_year / BASIC_PURCHASE_INFO['total_cash_required']) / BASIC_PURCHASE_INFO['total_cash_required']